In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip

^C


--2025-12-01 21:40:25--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-01T21%3A25%3A35Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-01T20%3A25%3A17Z&ske=2025-12-01T21%3A25%3A35Z&sks=b&skv=2018-11-09&sig=A7%2B2jjc%2FQT2HipuQX4qSVDmDJcoic46OYRqKSqM1Kzo%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDYyMzQyMiwibmJmIjoxNzY0NjIxNjIyLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi

In [ ]:
import zipfile

zip_path = "data.zip"
extract_to = "./extracted"

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_to)

print("Done!")


Done!


In [26]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [27]:
import torch
print(torch.__version__)

2.8.0+cpu


In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #input 3×200×200 output 32×198×198
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=0
        )
        
        # Max pooling --> 32×99×99
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # fc layer : flattened size = 32 * 99 * 99
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        # Convolution + ReLU
        x = F.relu(self.conv1(x))
        
        # Max pooling
        x = self.pool(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected + ReLU
        x = F.relu(self.fc1(x))
        
        # Output layer + Sigmoid
        x = torch.sigmoid(self.fc2(x))
        
        return x


In [29]:
model = SimpleCNN()
sample = torch.randn(1, 3, 200, 200)  # batch of 1
output = model(sample)
print(output.shape)


torch.Size([1, 1])


In [30]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)


In [31]:
criterion = nn.BCEWithLogitsLoss()


In [32]:
from torchsummary import summary
summary(model, input_size=(3, 200, 200))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------


In [33]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [34]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [35]:
train_dataset = datasets.ImageFolder(r"C:\Users\fulbe\OneDrive\Masaüstü\extra\MLZOOMCAMP\extracted\data\train", transform=train_transforms)
validation_dataset = datasets.ImageFolder(r"C:\Users\fulbe\OneDrive\Masaüstü\extra\MLZOOMCAMP\extracted\data\test", transform=test_transforms)


In [36]:
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)


In [37]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6932, Acc: 0.4875, Val Loss: 0.6779, Val Acc: 0.4876
Epoch 2/10, Loss: 0.6663, Acc: 0.4875, Val Loss: 0.6605, Val Acc: 0.4876
Epoch 3/10, Loss: 0.6537, Acc: 0.4875, Val Loss: 0.6672, Val Acc: 0.4876
Epoch 4/10, Loss: 0.6371, Acc: 0.4875, Val Loss: 0.6725, Val Acc: 0.4876
Epoch 5/10, Loss: 0.6299, Acc: 0.4875, Val Loss: 0.6601, Val Acc: 0.4876
Epoch 6/10, Loss: 0.6249, Acc: 0.4875, Val Loss: 0.6582, Val Acc: 0.4876
Epoch 7/10, Loss: 0.6140, Acc: 0.4875, Val Loss: 0.6638, Val Acc: 0.4876
Epoch 8/10, Loss: 0.6083, Acc: 0.4875, Val Loss: 0.6647, Val Acc: 0.4876
Epoch 9/10, Loss: 0.6001, Acc: 0.4875, Val Loss: 0.6705, Val Acc: 0.4876
Epoch 10/10, Loss: 0.5922, Acc: 0.4875, Val Loss: 0.6620, Val Acc: 0.4876


In [38]:
import numpy as np

# Training losses you provided
losses = np.array([
    0.6768, 0.6509, 0.6405, 0.6348, 0.6247,
    0.6199, 0.6138, 0.6059, 0.6006, 0.5947
])

# Training accuracies you provided
accuracies = np.array([
    0.4875, 0.4875, 0.4875, 0.4875, 0.4875,
    0.4875, 0.4875, 0.4875, 0.4888, 0.4875
])

# Median accuracy
median_acc = np.median(accuracies)

# Standard deviation of training loss
std_loss = np.std(losses)

print("Median accuracy:", median_acc)
print("Std of training loss:", std_loss)


Median accuracy: 0.4875
Std of training loss: 0.0238836010685156


In [40]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
])


In [ ]:
test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [43]:
train_dataset = datasets.ImageFolder(r"C:\Users\fulbe\OneDrive\Masaüstü\extra\MLZOOMCAMP\extracted\data\train", transform=train_transforms)
test_dataset = datasets.ImageFolder(r"C:\Users\fulbe\OneDrive\Masaüstü\extra\MLZOOMCAMP\extracted\data\test", transform=test_transforms)


In [44]:
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)


In [45]:
num_epochs = 10
test_losses = []

for epoch in range(num_epochs):
    # ---------------- TRAINING ----------------
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train

    # ---------------- TEST ----------------
    model.eval()
    test_running_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_running_loss += loss.item() * images.size(0)

    test_epoch_loss = test_running_loss / len(test_dataset)
    test_losses.append(test_epoch_loss)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f} | "
          f"Test Loss: {test_epoch_loss:.4f}")


Epoch 1/10 | Train Loss: 0.6570, Train Acc: 0.4875 | Test Loss: 0.6565
Epoch 2/10 | Train Loss: 0.6498, Train Acc: 0.4888 | Test Loss: 0.6634
Epoch 3/10 | Train Loss: 0.6568, Train Acc: 0.5050 | Test Loss: 0.6494
Epoch 4/10 | Train Loss: 0.6461, Train Acc: 0.4888 | Test Loss: 0.6507
Epoch 5/10 | Train Loss: 0.6447, Train Acc: 0.4925 | Test Loss: 0.6507
Epoch 6/10 | Train Loss: 0.6461, Train Acc: 0.5050 | Test Loss: 0.6556
Epoch 7/10 | Train Loss: 0.6399, Train Acc: 0.5000 | Test Loss: 0.6476
Epoch 8/10 | Train Loss: 0.6390, Train Acc: 0.4963 | Test Loss: 0.6503
Epoch 9/10 | Train Loss: 0.6429, Train Acc: 0.4975 | Test Loss: 0.6671
Epoch 10/10 | Train Loss: 0.6409, Train Acc: 0.4963 | Test Loss: 0.6465


In [46]:
import numpy as np
mean_test_loss = np.mean(test_losses)
print("Mean test loss:", mean_test_loss)


Mean test loss: 0.6537779734798924


In [47]:
import numpy as np

test_losses = np.array([0.6565, 0.6634, 0.6494, 0.6507, 0.6507,
                        0.6556, 0.6476, 0.6503, 0.6671, 0.6465])

mean_last5 = np.mean(test_losses[-5:])
print("Mean test loss (last 5 epochs):", mean_last5)


Mean test loss (last 5 epochs): 0.65342
